In [1]:
import sys
sys.path.append('../')
import src.config as config
from src.paths import PREPROCESSED_DATA_DIR


In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(current_date)

2024-03-03 00:00:00


In [3]:
df = pd.read_parquet(PREPROCESSED_DATA_DIR / 'full_data_2000_2005.parquet')

In [5]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(weeks=1)
def fetch_batch_data(df: pd.DataFrame, from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    df_filtered = df.copy()
    
    # we fetch raw data for the last 1 week, to add redundancy to our data pipeline
    years_shift = 23  # how many years to go back in time

    fetch_data_to_ = to_date.replace(year=to_date.year - years_shift)
    fetch_data_from_ = from_date.replace(year=from_date.year - years_shift)
    print(f'{fetch_data_from_=}, {fetch_data_to_=}')
    
    
    df_filtered = df_filtered[(df_filtered['full_date'] >= fetch_data_from_) & (df_filtered['full_date'] < fetch_data_to_)]

    return df_filtered


In [6]:
df_fetched = fetch_batch_data(df, fetch_data_from, fetch_data_to)

fetch_data_from_=Timestamp('2001-02-25 00:00:00'), fetch_data_to_=Timestamp('2001-03-03 00:00:00')


In [7]:
# add column with Unix epoch milliseconds
df_fetched['full_date_unix'] = df_fetched['full_date'].apply(lambda x: int(datetime.datetime.timestamp(x) * 1000))

In [8]:
# Checking for duplicates
df_fetched.duplicated(['full_date_unix','user','amount']).sum()

0

In [10]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Transaction data with RFM features at hourly frequency",
    primary_key = ['full_date_unix','user','amount'],
    event_time='full_date_unix',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/517268
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
feature_group.insert(df_fetched, write_options={"wait_for_job": True})


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/517268/fs/513091/fg/577898


Uploading Dataframe: 0.00% |          | Rows 0/3413 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: transactions_with_rfm_feature_group_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/517268/jobs/named/transactions_with_rfm_feature_group_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2a11f0ac3d0>, None)